Indicadores de Secretaría del Trabajo y Fomento al Empleo

In [1]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, janitor, stringr, readxl, writexl, tools,dplyr, purrr) 

Cargando paquete requerido: pacman

Warning message:
"package 'pacman' was built under R version 4.4.2"


DESCARGA Y LECTURA DE DATOS

In [2]:
#Crear carpeta para almacenar los microdatos descagados del INEGI
dir.create('Microdatos', showWarnings = FALSE)

#Crear carpetas para almacenar los resultados
dir.create('PGD', showWarnings = FALSE)

In [3]:
# Directorios principales
dir_datos <- 'Microdatos'

# Verificar si existen antes de intentar crearlos
if (!dir.exists(dir_datos)) {
    dir.create(dir_datos, recursive = TRUE)
}


# Verificar que los directorios realmente existen después de crearlos
print(file.exists(dir_datos))  # Debe devolver TRUE

[1] TRUE


In [4]:
descargar_enoe_csv <- function() {
    options(timeout = 90000)
    data_list <- list()

    descargar_extraer <- function(url, year_str, trim) {
        zip_path <- file.path('Microdatos', basename(url))
        download.file(url, zip_path, mode = 'wb')
        
        unzip_dir <- file.path('Microdatos', paste0('ENOE_', year_str, '_trim', trim))
        dir.create(unzip_dir, showWarnings = FALSE)
        unzip(zip_path, exdir = unzip_dir)

        unlink(zip_path) # Eliminar el archivo zip después de descomprimirlo
        csv_files <- list.files(unzip_dir, pattern = '\\.csv$', full.names = TRUE)
        
        if(length(csv_files) == 0) {
            warning('No se encontró el archivo CSV en ', unzip_dir)
            return(NULL) 
        }

        df_list <- list()

        for(csv_file in csv_files) {
            nombre <- file_path_sans_ext(basename(csv_file))
            tipo <- NA

            if (str_detect(nombre, 'COE1T?')) {
                tipo <- 'COE1'
                } else if (str_detect(nombre, 'COE2T?')) {
                    tipo <- 'COE2'
                    } else if (str_detect(nombre, 'HOGT?')) {
                        tipo <- 'HOG'
                        } else if (str_detect(nombre, 'SDEMT?')) {
                            tipo <- 'SDEM'
                            } else if (str_detect(nombre, 'VIVT?')) {
                                tipo <- 'VIV'
                                
                                }

            if(!is.na(tipo)) {
                df <- tryCatch({
                    read_csv(csv_file, show_col_types = FALSE) %>% clean_names()
                }, error = function(e) {
                    warning('Error al leer el archivo CSV de ', csv_file)
                    return(NULL) 
                })
                
                if(!is.null(df)) {
                    df_list[[tipo]] <- df
                }
            } 
                    
        }
        
    return(df_list)
    
    }
    
    #iterar los años y trimestres de descarga
    for(year in 18:20) {
    year_str <- paste0('20', year)
    data_list[[year_str]] <- list()
    
    for(trim in 1:4) {
      url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/20', year, 'trim', trim, '_csv.zip')
      
      df <- descargar_extraer(url, year_str, trim)
      
      if(!is.null(df)) {
        data_list[[year_str]][[paste0('trim', trim)]] <- df
      }
    }
  }

    for(year in 21:22) {
        year_str <- paste0('20', year)
        data_list[[year_str]] <- list()
        
        for(trim in 1:4) {
        url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_20', year, '_trim', trim, '_csv.zip')
        
        df <- descargar_extraer(url, year_str, trim)

        if(!is.null(df)) {
            data_list[[year_str]][[paste0('trim', trim)]] <- df
        }
    }

    }
    
    for(year in 23:24) {
        year_str <- paste0('20', year)
        data_list[[year_str]] <- list()
        
        for(trim in 1:4) {
        url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_20', year, '_trim', trim, '_csv.zip')
                
        df <- descargar_extraer(url, year_str, trim)
        
        if(!is.null(df)) {
            data_list[[year_str]][[paste0('trim', trim)]] <- df
        }
    
    }

    } 
    
    save(data_list, file = 'Microdatos/data_list.RData')


    return(data_list)

}

In [ ]:
invisible(descargar_enoe_csv())

In [5]:
#Cargar la lista de datos guardada para no tener que descargarla nuevamente 
load("Microdatos/data_list.RData")

In [ ]:
#Confirmar que la estructura de las listas de datos estén bien
str(data_list$'2023'$'trim4'$'COE1')

Población informal en la Ciudad de México

In [7]:

# Definir los años y trimestres a trabajar
anios <- as.character(2021:2024)
trimestres <- paste0("trim", 1:4)

# Extraer y unir SDEM de todos los trimestres y años, estandarizando columnas a character
sdem_todos <- cross_df(list(anio = anios, trim = trimestres)) %>%
  mutate(data = map2(anio, trim, ~ {
    df <- data_list[[.x]][[.y]][["SDEM"]]
    
    if (!is.null(df)) {
      df <- df %>%
        mutate(across(everything(), as.character)) %>%  #todas las columnas como character
        mutate(anio = as.integer(.x), trimestre = .y)
    } else {
      NULL
    }
  })) %>%
  pull(data) %>%
  compact() %>%
  bind_rows()

Warning message:
"`cross_df()` was deprecated in purrr 1.0.0.
ℹ Please use `tidyr::expand_grid()` instead.
ℹ See <https://github.com/tidyverse/purrr/issues/768>."


In [8]:
sdem_todos <- sdem_todos %>%
  mutate(
    fac_tri = as.numeric(fac_tri),
    eda = as.integer(eda),
    ent = as.integer(ent)
  )

In [9]:
Variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Informal <- list() #Lista para vaciar los valores de población ocupada en el sector informal

In [10]:
calculo_informal <- function(data_list) {
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next

#calcular la población ocupada wn la cdmx  :) 
      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98) %>%
        filter(clase1 == 1, ent == 9) 
      
      # Guardar nombres y df completo por si se necesita
      Variables[[paste0("v", year, "_", trim)]] <<- names(df)
      PobTot[[paste0("pob_total_", year, "_", trim)]] <<- df



     # Calcular la población ocupada en el sector informal
   df_informal <- df %>%
        filter(emp_ppal == 1) %>%
        filter( (pos_ocu == 1 & seg_soc == 2) |
          (pos_ocu %in% c(2, 3, 4) & tue2 %in% c( 2, 5, 6))) %>%
        mutate(sector = "Informal")



      total_sector <- sum(df_informal$fac_tri, na.rm = TRUE)
      total_pob <- sum(df$fac_tri, na.rm = TRUE)
      porcentaje <- (total_sector / total_pob) * 100

      Informal[[paste0("informal_", year, "_", trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        sector = "Informal",
        total_sector = total_sector,
        pob_total = total_pob,
        porcentaje = porcentaje
      )
    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_informal(data_list))


In [11]:
# Unir resultados finales
informal_final <- bind_rows(Informal) %>%
  mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
  arrange(year, trimestre)

# Agregar por año
informal_anual <- informal_final %>%
  group_by(year, sector) %>%
  summarise(
    total_sector = sum(total_sector, na.rm = TRUE),
    pob_total = sum(pob_total, na.rm = TRUE),
    porcentaje = (total_sector / pob_total) * 100,
    .groups = "drop"
  ) %>%
  arrange(year)


informal_anual <- informal_final %>%
  group_by(year, sector) %>%
  summarise(
    total_sector = sum(total_sector, na.rm = TRUE),
    pob_total = sum(pob_total, na.rm = TRUE),
    porcentaje = (total_sector / pob_total) * 100,
    .groups = "drop"
  ) %>%
  arrange(year)

# Mostrar resultado anual
print(informal_anual)



# A tibble: 5 × 5
   year sector   total_sector pob_total porcentaje
  <int> <chr>           <dbl>     <dbl>      <dbl>
1  2021 Informal      7957422  18084026       44.0
2  2022 Informal      8400414  19170899       43.8
3  2023 Informal      8560864  19559988       43.8
4  2024 Informal      8666509  20262721       42.8
5  2025 Informal      2143968   5066685       42.3


In [12]:
#CALCULO CORRECTO 

tpo4 <- list()

calculo_tpo4 <- function(data_list) {
  
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next
      
      df_base <- data_list[[year]][[trim]][['SDEM']] %>%
        janitor::clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(
          r_def == 0,
          c_res %in% c(1, 3),
          edad >= 15 & edad <= 98,
          ent == 9,
          clase2 == 1  # Solo ocupados
        )

      # Total población ocupada
      total_ocupados <- df_base %>%
        summarise(total = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total)

      # Aplicar filtro TPO4
      df_tpo4 <- df_base %>%
        filter(
          (tip_con == 5 & seg_soc == 2 & emp_ppal == 2) |
          (pos_ocu %in% c(1, 2, 3, 4) & emp_ppal == 1)
        )

      total_tpo4 <- sum(df_tpo4$fac_tri, na.rm = TRUE)

      porcentaje <- (total_tpo4 / total_ocupados) * 100

      tpo4[[paste0('promedio_', year, '_', trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        total_ocupados = total_ocupados,
        total_tpo4 = total_tpo4,
        porcentaje = porcentaje
      )
    }
  }
}

# Ejecutar función
invisible(calculo_tpo4(data_list))

# Combinar resultados
tpo4_final <- bind_rows(tpo4) %>%
  mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
  arrange(year, trimestre)

print(tpo4_final)


# A tibble: 17 × 5
    year trimestre total_ocupados total_tpo4 porcentaje
   <int> <fct>              <dbl>      <dbl>      <dbl>
 1  2021 trim1            3870454    1794589       46.4
 2  2021 trim2            4117427    1939674       47.1
 3  2021 trim3            4393859    2168037       49.3
 4  2021 trim4            4465106    2144407       48.0
 5  2022 trim1            4385718    2023263       46.1
 6  2022 trim2            4534376    2180962       48.1
 7  2022 trim3            4546515    2168004       47.7
 8  2022 trim4            4698174    2152425       45.8
 9  2023 trim1            4693298    2169763       46.2
10  2023 trim2            4709095    2179459       46.3
11  2023 trim3            4712506    2191013       46.5
12  2023 trim4            4657701    2131762       45.8
13  2024 trim1            4715069    2192253       46.5
14  2024 trim2            4834239    2190661       45.3
15  2024 trim3            4894634    2215259       45.3
16  2024 trim4            501

Tasa de participación económica de las mujeres                                                                                                                                                                              Porcentaje que representa la población económicamente activa (PEA) respecto a la de 15 y más años de edad.


In [13]:
Variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
PEAmujeres <- list() #Lista para vaciar los valores de población eco

In [14]:
# Lista para resultados
TPE_Mujeres <- list()

In [15]:
calculo_tpe_mujeres <- function(data_list) {
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next
      
      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda),
               sexo = as.integer(sex)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98,
               sex == 2,  
               ent == 9)   

      # Población total mujeres 15+
      pob_total_muj <- sum(df$fac_tri, na.rm = TRUE)
      
      #pea mujeres
      pea_muj <- df %>%
        filter(clase1 == 1) %>%
        summarise(total_pea = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_pea)
      
      tpe <- (pea_muj / pob_total_muj) * 100
      
      TPE_Mujeres[[paste0("tpe_", year, "_", trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        pob_total_muj = pob_total_muj,
        pea_muj = pea_muj,
        tpe_muj = tpe
      )
    }
  }
}

In [16]:
# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_tpe_mujeres(data_list))

In [17]:
# Unir resultados finales
tpe_mujeres_final <- bind_rows(TPE_Mujeres) %>%
  mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
  arrange(year, trimestre)

In [18]:
# Mostrar resultados
print(tpe_mujeres_final)

# A tibble: 17 × 5
    year trimestre pob_total_muj pea_muj tpe_muj
   <int> <fct>             <dbl>   <dbl>   <dbl>
 1  2021 trim1           4168688 1811826    43.5
 2  2021 trim2           4151850 1924527    46.4
 3  2021 trim3           4174088 2126013    50.9
 4  2021 trim4           4152856 2114965    50.9
 5  2022 trim1           4148738 2055301    49.5
 6  2022 trim2           4163416 2095642    50.3
 7  2022 trim3           4214921 2124193    50.4
 8  2022 trim4           4205491 2220223    52.8
 9  2023 trim1           4171434 2152638    51.6
10  2023 trim2           4239135 2246701    53.0
11  2023 trim3           4202125 2185798    52.0
12  2023 trim4           4275915 2199966    51.5
13  2024 trim1           4339569 2272205    52.4
14  2024 trim2           4355221 2378223    54.6
15  2024 trim3           4349180 2386025    54.9
16  2024 trim4           4354909 2484036    57.0
17  2025 trim1           4295927 2380749    55.4


Tasa de Condiciones Críticas de Ocupación 

In [19]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, janitor, stringr, readxl, writexl, tools)

In [20]:
dir.create('Microdatos', showWarnings = FALSE)

#Crear carpetas para almacenar los resultados por visor del sistema
dir.create('PGD', showWarnings = FALSE)
dir.create('Visor_CDMX', showWarnings = FALSE) 

In [21]:
# Directorios principales
dir_datos <- 'Microdatos'

# Verificar si existen antes de intentar crearlos
if (!dir.exists(dir_datos)) {
  dir.create(dir_datos, recursive = TRUE)
}


# Verificar que los directorios realmente existen después de crearlos
print(file.exists(dir_datos))  # Debe devolver TRUE

[1] TRUE


In [22]:
descargar_enoe_csv <- function() {
  options(timeout = 90000)
  data_list <- list()
  
  descargar_extraer <- function(url, year_str, trim) {
    zip_path <- file.path('Microdatos', basename(url))
    download.file(url, zip_path, mode = 'wb')
    
    unzip_dir <- file.path('Microdatos', paste0('ENOE_', year_str, '_trim', trim))
    dir.create(unzip_dir, showWarnings = FALSE)
    unzip(zip_path, exdir = unzip_dir)
    
    unlink(zip_path) # Eliminar el archivo zip después de descomprimirlo
    csv_files <- list.files(unzip_dir, pattern = '\\.csv$', full.names = TRUE)
    
    if(length(csv_files) == 0) {
      warning('No se encontró el archivo CSV en ', unzip_dir)
      return(NULL) 
    }
    
    df_list <- list()
    
    for(csv_file in csv_files) {
      nombre <- file_path_sans_ext(basename(csv_file))
      tipo <- NA
      
      if (str_detect(nombre, 'COE1T?')) {
        tipo <- 'COE1'
      } else if (str_detect(nombre, 'COE2T?')) {
        tipo <- 'COE2'
      } else if (str_detect(nombre, 'HOGT?')) {
        tipo <- 'HOG'
      } else if (str_detect(nombre, 'SDEMT?')) {
        tipo <- 'SDEM'
      } else if (str_detect(nombre, 'VIVT?')) {
        tipo <- 'VIV'
        
      }
      
      if(!is.na(tipo)) {
        df <- tryCatch({
          read_csv(csv_file, show_col_types = FALSE) %>% clean_names()
        }, error = function(e) {
          warning('Error al leer el archivo CSV de ', csv_file)
          return(NULL) 
        })
        
        if(!is.null(df)) {
          df_list[[tipo]] <- df
        }
      } 
      
    }
    
    return(df_list)
    
  }
  
  #iterar los años y trimestres de descarga    
  for(year in 21:22) {
    year_str <- paste0('20', year)
    data_list[[year_str]] <- list()
    
    for(trim in 1:4) {
      url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_20', year, '_trim', trim, '_csv.zip')
      
      df <- descargar_extraer(url, year_str, trim)
      
      if(!is.null(df)) {
        data_list[[year_str]][[paste0('trim', trim)]] <- df
      }
    }
    
  }
  
  for(year in 23:25) {
    year_str <- paste0('20', year)
    data_list[[year_str]] <- list()
    
    for(trim in 1:4) {
      url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_20', year, '_trim', trim, '_csv.zip')
      
      df <- descargar_extraer(url, year_str, trim)
      
      if(!is.null(df)) {
        data_list[[year_str]][[paste0('trim', trim)]] <- df
      }
      
    }
    
  } 
  
  save(data_list, file = 'Microdatos/data_list.RData')
  
  
  return(data_list)
  
}

In [23]:
invisible(descargar_enoe_csv())

Warning message in download.file(url, zip_path, mode = "wb"):
"downloaded length 12779079 != reported length 24586788"
Warning message in download.file(url, zip_path, mode = "wb"):
"URL 'https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2021_trim1_csv.zip': status was 'Failure when receiving data from the peer'"


ERROR: Error in download.file(url, zip_path, mode = "wb"): download from 'https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_2021_trim1_csv.zip' failed


In [25]:
load("Microdatos/data_list.RData")

In [ ]:
#Confirmar que la estructura de las listas de datos estén bien
str(data_list$'2024'$'trim4'$'SDEM')

Función para calcular población ocupada

In [31]:
po <- list()

In [32]:
#Define la función 'calculo_pea' que toma como argumento una lista de datos (data_list)
calculo_po <- function(data_list) {
  
  # Itera sobre los años contenidos en la lista
  for (year in names(data_list)) {
    
    # Itera sobre los trimestres contenidos dentro de cada año
    for (trim in names(data_list[[year]])) {
      
      # Verifica que exista el elemento 'SDEM' dentro del trimestre actual; si no, pasa al siguiente
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next
      
      # Extrae el dataframe 'SDEM', estandariza nombres, convierte edad a numérico y filtra:
      # personas con respuesta definitiva (r_def == 0), residentes habituales (c_res 1 o 3),
      # entre 15 y 98 años, y que pertenecen a la entidad 9 (CDMX)
      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98, ent == 9)
      
      # Filtra a la población ocupada (clase2 == 1),
      # y calcula el total ponderado usando 'fac_tri'
      po <- df %>%
        filter(clase2 == 1) %>%
        summarise(total_po = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_po)
      
      # Guarda el resultado en una lista global llamada 'PEA' usando el nombre 'promedio_año_trimestre'
      po[[paste0('promedio_', year, '_', trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        total_po = po
      )
    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_po(data_list))

In [33]:
po_final <- bind_rows(po) %>%
  mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
  arrange(year, trimestre) 

In [34]:
print(po_final)

# A tibble: 17 × 3
    year trimestre total_po
   <int> <fct>        <dbl>
 1  2021 trim1      3870454
 2  2021 trim2      4117427
 3  2021 trim3      4393859
 4  2021 trim4      4465106
 5  2022 trim1      4385718
 6  2022 trim2      4534376
 7  2022 trim3      4546515
 8  2022 trim4      4698174
 9  2023 trim1      4693298
10  2023 trim2      4709095
11  2023 trim3      4712506
12  2023 trim4      4657701
13  2024 trim1      4715069
14  2024 trim2      4834239
15  2024 trim3      4894634
16  2024 trim4      5010045
17  2025 trim1      4892875


Cálculo con variable TCCO

In [36]:
tpo1 <- list()

In [37]:
# PO que trabajan menos de 35 hrs a la semana por razones de mercado 
# Define la función 'calculo_pea' que toma como argumento una lista de datos (data_list)
calculo_tpo1 <- function(data_list) {
  
  # Itera sobre los años contenidos en la lista
  for (year in names(data_list)) {
    
    # Itera sobre los trimestres contenidos dentro de cada año
    for (trim in names(data_list[[year]])) {
      
      # Verifica que exista el elemento 'SDEM' dentro del trimestre actual; si no, pasa al siguiente
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next
      
      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98, ent == 9, tcco == 1)
      
      tpo1 <- df %>%
        filter(clase2 == 1,) %>%
        summarise(total_tpo1 = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_tpo1)
      
      # Guarda el resultado en una lista global llamada 'PEA' usando el nombre 'promedio_año_trimestre'
      tpo1[[paste0('promedio_', year, '_', trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        total_tpo1 = tpo1
      )
    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_tpo1(data_list))

In [38]:
tpo1_final <- bind_rows(tpo1) %>%
  mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
  arrange(year, trimestre) 

print(tpo1_final)

# A tibble: 17 × 3
    year trimestre total_tpo1
   <int> <fct>          <dbl>
 1  2021 trim1         131874
 2  2021 trim2          82616
 3  2021 trim3          90823
 4  2021 trim4          59882
 5  2022 trim1          23922
 6  2022 trim2          20805
 7  2022 trim3           1894
 8  2022 trim4          13949
 9  2023 trim1           3350
10  2023 trim2           7612
11  2023 trim3           9134
12  2023 trim4           5434
13  2024 trim1           7560
14  2024 trim2          10564
15  2024 trim3          10102
16  2024 trim4           5108
17  2025 trim1           1588


PO que trabajan más de 35 hrs a la semana con ingresos mensuales inferiores al salario mínimo 

In [39]:
tpo2 <- list()

Función para calcular población económicamente activa

In [40]:
# Define la función 'calculo_pea' que toma como argumento una lista de datos (data_list)
calculo_tpo2 <- function(data_list) {
  
  # Itera sobre los años contenidos en la lista
  for (year in names(data_list)) {
    
    # Itera sobre los trimestres contenidos dentro de cada año
    for (trim in names(data_list[[year]])) {
      
      # Verifica que exista el elemento 'SDEM' dentro del trimestre actual; si no, pasa al siguiente
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next
      
      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98, ent == 9, tcco == 2)
      
      tpo2 <- df %>%
        filter(clase2 == 1,) %>%
        summarise(total_tpo2 = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_tpo2)
      
      tpo2[[paste0('promedio_', year, '_', trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        total_tpo2 = tpo2
      )
    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_tpo2(data_list))

In [41]:
tpo2_final <- bind_rows(tpo2) %>%
  mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
  arrange(year, trimestre) 

print(tpo2_final)

# A tibble: 17 × 3
    year trimestre total_tpo2
   <int> <fct>          <dbl>
 1  2021 trim1         563679
 2  2021 trim2         672696
 3  2021 trim3         667223
 4  2021 trim4         701978
 5  2022 trim1         903430
 6  2022 trim2         943887
 7  2022 trim3         935990
 8  2022 trim4         851194
 9  2023 trim1        1073442
10  2023 trim2         988900
11  2023 trim3        1050679
12  2023 trim4         961403
13  2024 trim1        1050374
14  2024 trim2        1009082
15  2024 trim3        1041935
16  2024 trim4         920596
17  2025 trim1        1068465


PO que trabajan más de 48 hrs a la semana ganando hasta dos salarios mínimos


In [42]:
tpo3 <- list()

Función para calcular población económicamente activa

In [43]:
# Define la función 'calculo_pea' que toma como argumento una lista de datos (data_list)
calculo_tpo3 <- function(data_list) {
  
  # Itera sobre los años contenidos en la lista
  for (year in names(data_list)) {
    
    # Itera sobre los trimestres contenidos dentro de cada año
    for (trim in names(data_list[[year]])) {
      
      # Verifica que exista el elemento 'SDEM' dentro del trimestre actual; si no, pasa al siguiente
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next
      
      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98, ent == 9, tcco == 3)
      
      tpo3 <- df %>%
        filter(clase2 == 1,) %>%
        summarise(total_tpo3 = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_tpo3)
      
      tpo3[[paste0('promedio_', year, '_', trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        total_tpo3 = tpo3
      )
    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_tpo3(data_list))

In [44]:
tpo3_final <- bind_rows(tpo3) %>%
  mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
  arrange(year, trimestre) 

print(tpo3_final)

# A tibble: 17 × 3
    year trimestre total_tpo3
   <int> <fct>          <dbl>
 1  2021 trim1         321314
 2  2021 trim2         385308
 3  2021 trim3         467880
 4  2021 trim4         446623
 5  2022 trim1         437399
 6  2022 trim2         503324
 7  2022 trim3         446272
 8  2022 trim4         508024
 9  2023 trim1         450708
10  2023 trim2         455084
11  2023 trim3         466355
12  2023 trim4         518503
13  2024 trim1         492740
14  2024 trim2         439753
15  2024 trim3         476097
16  2024 trim4         466626
17  2025 trim1         438933


In [45]:
tpo_datos <- po_final %>%
  # Unir con po1_final por 'year' y 'trimestre'
  left_join(select(tpo1_final, year, trimestre, total_tpo1), by = c("year", "trimestre")) %>%
  # Unir con po2_final
  left_join(select(tpo2_final, year, trimestre, total_tpo2), by = c("year", "trimestre")) %>%
  # Unir con po3_final
  left_join(select(tpo3_final, year, trimestre, total_tpo3), by = c("year", "trimestre"))

# Visualizar los primeros registros del nuevo data frame combinado
head(tpo_datos)

year,trimestre,total_po,total_tpo1,total_tpo2,total_tpo3
<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
2021,trim1,3870454,131874,563679,321314
2021,trim2,4117427,82616,672696,385308
2021,trim3,4393859,90823,667223,467880
2021,trim4,4465106,59882,701978,446623
2022,trim1,4385718,23922,903430,437399
2022,trim2,4534376,20805,943887,503324


In [48]:
tpo_datos <- tpo_datos %>%
  mutate(
    suma = total_tpo1 + total_tpo2 + total_tpo3,
    indicador = (suma / total_po) * 100
  )

head(tpo_datos)

year,trimestre,total_po,total_tpo1,total_tpo2,total_tpo3,suma,indicador
<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021,trim1,3870454,131874,563679,321314,1016867,26.27255
2021,trim2,4117427,82616,672696,385308,1140620,27.70225
2021,trim3,4393859,90823,667223,467880,1225926,27.90090
2021,trim4,4465106,59882,701978,446623,1208483,27.06505
2022,trim1,4385718,23922,903430,437399,1364751,31.11807
2022,trim2,4534376,20805,943887,503324,1468016,32.37526


Índice de productividad 

Relación(cociente) entre el índice de valor agregado respecto del índice de personas ocupadas. (Metodología de INEGI(2019) del índice de global de productividad laboral de la economía por personas ocupadas)

In [4]:
#Se cargan las paqueterías y librerías
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, janitor, stringr, readxl, writexl, tools, dplyr, jsonlite)

Calculamos el Índice del Valor Agregado Bruto de la Ciudad de México
Los datos que se ocuparán serán extraídos desde la API de banco de indicadores del INEGI, la cual se puede conseultar aquí: https://www.inegi.org.mx/servicios/api_indicadores.html . Para su uso, se debe generar un token, el cual es personal. Una vez generado, deberá introducir en la siguiente parte del código. 


In [ ]:
#Introducir token generado por la API de INEGI (es personal)
token <-  "¡Coloca aquí tu Toke! :D"
#construye la URL 
url <- paste0("https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/746172/es/0700/false/BIE/2.0/", token, "?type=json")

url

[1] "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/746172/es/0700/false/BIE/2.0/c0199ed6-cc32-7f4c-d1f8-7ceea1894695?type=json"

In [10]:
json3 <- fromJSON(url)
series <- as.data.frame(json3$Series)
observations <- bind_rows(series$OBSERVATIONS, .id = "parent_row")
datos_vac <- observations %>% select(TIME_PERIOD, OBS_VALUE)

Se descargan y transforman los datos

¡Se ordenan y filtran los datos!
Se realiza el cálculo del índice del Valor Agregado para la Ciudad de México, el cual corresponde al cociente del Valor Agregado a precios constantes (de 2018) entre el promedio anual del Valor Agregado en el año base.

In [11]:
datos_vac <- datos_vac %>%
  arrange(TIME_PERIOD) %>%
  rename("año" = TIME_PERIOD) %>%
  rename("va"=OBS_VALUE) %>%
  mutate(va = as.numeric(va)) %>%
  filter(año %in% c(2018:2025)) %>%
  mutate(base = first(va)) %>%
  mutate(iva = (va/base)*100)  #Se calcula el índice de valor agregado

In [12]:
head(datos_vac)

,año,va,base,iva
,<chr>,<dbl>,<dbl>,<dbl>
1,2018,3482680,3482680,100.00000
2,2019,3468776,3482680,99.60077
3,2020,3112253,3482680,89.36374
4,2021,3297839,3482680,94.69255
5,2022,3434238,3482680,98.60905
6,2023,3581342,3482680,102.83292


Cálculo del Índice del Producto Interno Bruto (IPIB). 
Se descargan los datos. Posteriormente, se calcula el índice, el cual es igual al PIB a precios constantes (base 2018) en el trimestre (para nivel entidad es anual) entre el promedio anual del PIB total en el año base (2018).

In [ ]:
token <- "¡Coloca aquí tu Toke! :D"
url <- paste0("https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/746106/es/0700/false/BIE/2.0/", token, "?type=json")

Se extraen los datos que se requieren

In [14]:
# Descarga y transforma los datos
datos_json <- fromJSON(url)
series <- datos_json$Series
observations <- bind_rows(series$OBSERVATIONS, .id = "parent_row")
datos_pib <- observations %>% select(TIME_PERIOD, OBS_VALUE)

In [15]:
# Vista previa
head(datos_pib)

,TIME_PERIOD,OBS_VALUE
,<chr>,<chr>
1,2023,3806082.926
2,2022,3650609.372
3,2021,3496055.811
4,2020,3293855.308
5,2019,3679917.346
6,2018,3694574.874


Se ordenan y filtran los datos. Y se calcula el IPIB

In [16]:
datos_pib <- datos_pib %>%
  arrange(TIME_PERIOD) %>%
  rename("año" = TIME_PERIOD) %>%
  rename("pibe"=OBS_VALUE) %>%
  mutate(pibe = as.numeric(pibe)) %>%
  filter(año %in% c(2018:2025)) %>%
  mutate(base = first(pibe)) %>%
  mutate(ipibe = (pibe/base)*100) %>%
  rename("a" = año)

head(datos_pib) 

,a,pibe,base,ipibe
,<chr>,<dbl>,<dbl>,<dbl>
1,2018,3694575,3694575,100.00000
2,2019,3679917,3694575,99.60327
3,2020,3293855,3694575,89.15384
4,2021,3496056,3694575,94.62674
5,2022,3650609,3694575,98.81000
6,2023,3806083,3694575,103.01816


Se unen las datas del IPIB e IVA para nivel CDMX. 

In [22]:
datos_ciudad <- cbind(datos_pib, datos_vac) %>%
  select(año, ipibe, iva)
head(datos_ciudad)  

,año,ipibe,iva
,<chr>,<dbl>,<dbl>
1,2018,100.00000,100.00000
2,2019,99.60327,99.60077
3,2020,89.15384,89.36374
4,2021,94.62674,94.69255
5,2022,98.81000,98.60905
6,2023,103.01816,102.83292


Se calcula el índice de la Población Ocupada de la Ciudad de México

El IPO, corresponde a la división del total de la población ocupada en el año n, sobre el promedio anual de la población ocupada en el año base (2018). Para lo cual, se descargan los datos de la ENOE 2018-2023.

In [23]:
# Ruta base (ajusta a tu ruta real)
carpeta_base <- "Microdatos"
ruta_base <- paste0(getwd(), "/", carpeta_base)


# Listar subcarpetas
carpetas <- list.dirs(ruta_base, recursive = FALSE, full.names = TRUE)

data_list <- list()


for (ruta in carpetas) {
  nombre_carpeta <- basename(ruta)
  
  # Extraer año y trimestre
  if (str_detect(nombre_carpeta, "\\d{4}_trim\\d")) {
    año <- str_extract(nombre_carpeta, "\\d{4}")
    trim <- str_extract(nombre_carpeta, "trim\\d")
    
    # Buscar archivos CSV que contengan 'sdem' o 'SDEM'
    archivos_sdem <- list.files(ruta, pattern = "(?i)sdem.*\\.csv$", full.names = TRUE)
    
    # Leer todos los archivos encontrados
    if (length(archivos_sdem) > 0) {
      tablas <- lapply(archivos_sdem, read.csv)
      
      # Crear año si no existe
      if (!año %in% names(data_list)) {
        data_list[[año]] <- list()
      }
      
      # Si hay uno solo, guardamos como data.frame; si hay varios, como lista de data.frames
      data_list[[año]][[trim]] <- if (length(tablas) == 1) tablas[[1]] else tablas
    }
  }
}






calculo_po <- function(data_list, filtro_entidad = NaN, resultado_lista = list()) {
  
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      
      df_raw <- data_list[[year]][[trim]]
      if (!is.data.frame(df_raw)) next
      
      df <- df_raw %>%
        janitor::clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(
          r_def == 0,
          c_res %in% c(1, 3),
          edad >= 15 & edad <= 98
        )
      
      # Filtrar por entidad si se especifica
      if (!is.nan(filtro_entidad)) {
        df <- df %>% filter(ent == filtro_entidad)
      }
      
      # Detectar el nombre del factor de expansión
      factor_exp <- if ("fac_tri" %in% names(df)) "fac_tri" else "fac"
      
      # Calcular población ocupada
      total_po <- df %>%
        filter(clase2 == 1) %>%
        summarise(total_po = sum(.data[[factor_exp]], na.rm = TRUE)) %>%
        pull(total_po)
      
      # Guardar en la lista que se pasó como parámetro
      resultado_lista[[paste0("promedio_", year, "_", trim)]] <- tibble(
        year = as.integer(year),
        trimestre = trim,
        total_po = total_po
      )
    }
  }
  
  return(resultado_lista)
}
po_nacional <- calculo_po(data_list)

In [24]:
# Unir los resultados
po_nalfin <- bind_rows(po_nacional) %>%
  mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
  arrange(year, trimestre)

po_cdmx <- calculo_po(data_list,filtro_entidad = 9)

# Unir los resultados
po_cdmxfin <- bind_rows(po_cdmx) %>%
  mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
  arrange(year, trimestre)

In [25]:
po_cdmxfin

year,trimestre,total_po
<int>,<fct>,<int>
2018,trim1,4387044
2018,trim2,4473314
2018,trim3,4473679
2018,trim4,4411790
2019,trim1,4452453
2019,trim2,4505698
2019,trim3,4516175
2019,trim4,4507009
2020,trim1,4439595


El cálculo anterior, corresponde al total de la población ocupada de forma trimestral. Por lo tanto, ahora se deberá calcular de forma anual, ya que los datos del IVA e IPIB son anuales. 

In [26]:
ipo_cdmxfin <- po_cdmxfin %>%
  group_by(year) %>%
  mutate(prom_po = mean(total_po, na.rm = TRUE)) %>%
  ungroup() %>%
  mutate(base = first(prom_po)) %>%
  mutate (ipo = (total_po/base)*100)

#ipo anual cdmx
ipo_cdmx_anual <- ipo_cdmxfin %>%
  group_by(year) %>%
  summarise(prom_ipo = mean(ipo, na.rm = TRUE)) %>%
  ungroup() %>%
  rename("ipo" = prom_ipo) %>%
  filter(year %in% c(2018:2023))
  
print(ipo_cdmx_anual)

# A tibble: 6 × 2
   year   ipo
  <int> <dbl>
1  2018 100  
2  2019 101. 
3  2020 100. 
4  2021  94.9
5  2022 102. 
6  2023 106. 


Se unen las tablas de datos (IVA, IPIB e IPO) para calcular el IPL de la Ciudad de México. 
Baándonos en su cálculo respecto a la población ocupada, Existen dos formas de calcularlo: 
1.- (IPIB/IPO) * 100
2.- (IVA/IPO) * 100
Se realizarán las dos posibilidades. 

In [27]:
IPL_ciudad <- cbind(datos_ciudad, ipo_cdmx_anual) %>%
  select(año, ipibe, iva, ipo) %>%
  mutate(ipl = (iva/ipo)*100) %>%
  mutate(ipl_pib = (ipibe/ipo)*100)
print(IPL_ciudad)

   año     ipibe       iva       ipo       ipl   ipl_pib
1 2018 100.00000 100.00000 100.00000 100.00000 100.00000
2 2019  99.60327  99.60077 101.32712  98.29626  98.29873
3 2020  89.15384  89.36374 100.07074  89.30057  89.09082
4 2021  94.62674  94.69255  94.93413  99.74553  99.67621
5 2022  98.81000  98.60905 102.36087  96.33471  96.53103
6 2023 103.01816 102.83292 105.78600  97.20844  97.38355


In [28]:
head(IPL_ciudad)    

,año,ipibe,iva,ipo,ipl,ipl_pib
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2018,100.00000,100.00000,100.00000,100.00000,100.00000
2,2019,99.60327,99.60077,101.32712,98.29626,98.29873
3,2020,89.15384,89.36374,100.07074,89.30057,89.09082
4,2021,94.62674,94.69255,94.93413,99.74553,99.67621
5,2022,98.81000,98.60905,102.36087,96.33471,96.53103
6,2023,103.01816,102.83292,105.78600,97.20844,97.38355
